In [9]:
import pandas as pd
import copy

In [16]:
data = pd.read_csv('state_data_2015-2019.csv')

C:\Users\andyw\AppData\Local\Temp\ipykernel_8296\2425668755.py:1: DtypeWarning: Columns (21,22,23,24,25,26,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,48,49,50,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,96,97,98,99,100,101,102,103,104,105,106,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,242,245,246,247,248,249,250,251,252,253,254,255,256,257,258,260) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('state_data_2015-2019.csv')


In [17]:
# dropping categorical columns that are already dichotomonized

columns_to_be_dropped = [q for q in data.columns if q[0] == 'q' and q[1] != 'n']
data = data.drop(columns = columns_to_be_dropped)


In [18]:
# dropping unnecesary columns

data = data.drop(columns=['sitename','sitetype','sitetypenum','year','survyear','record'])

In [19]:
# using race7 only

data = data.drop(columns = ['race4'])

In [20]:
# dropping observations where the suicide attempt column is not present

data = data.dropna(subset = ['qn28'])

In [21]:
predictors = data.drop(columns = ['qn26','qn27','qn28','qn29'], inplace = False)

suicide_attempt = (data['qn28'] == 1).astype(int)



In [22]:
data

,sitecode,weight,stratum,PSU,age,sex,grade,race7,stheight,stweight,...,qnwenthungry,qnmusclestrength,qnsunscreenuse,qnindoortanning,qnsunburn,qnconcentrating,qncurrentasthma,qnwheresleep,qnspeakenglish,qntransgender
4,Alaska,30.7774,14,3,15 years old,NaN,10th,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
5,Alaska,23.3834,31,2,17 years old,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
6,Alaska,19.1526,5,6,18 years old or older,NaN,12th,Hispanic/Latino,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
7,Alaska,19.2439,2,6,NaN,Male,11th,Black or African American,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
8,Alaska,39.2016,28,2,NaN,Male,12th,Am Indian / Alaska Native,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576446,West Virginia,68.4095,13,1,18 years old or older,Female,11th,White,1.73,68.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
576447,West Virginia,87.0022,13,1,18 years old or older,Female,12th,White,1.75,90.72,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
576448,West Virginia,157.7903,15,1,18 years old or older,Female,12th,White,1.63,79.38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
576450,West Virginia,157.7903,15,1,18 years old or older,Female,12th,White,1.63,71.22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:

data_processed_for_training = data.iloc[:,0:4]

for column in predictors.columns[4:9]:
    dummy_columns = pd.get_dummies(predictors[column], prefix = column)
    dummy_columns = dummy_columns.drop(columns = [column + '_NaN'], errors = 'ignore')
    data_processed_for_training = data_processed_for_training.join(dummy_columns)

for column in predictors.columns[9:13]:
    data_processed_for_training = data_processed_for_training.join(predictors[column])

for column in predictors.columns[13:]:
    dummy_columns = pd.get_dummies(predictors[column], prefix = column)
    dummy_columns = dummy_columns.drop(columns = [column + '_NaN'], errors = 'ignore')
    data_processed_for_training = data_processed_for_training.join(dummy_columns)

data_processed_for_training = data_processed_for_training.fillna(0)

data_processed_for_training = data_processed_for_training.join(suicide_attempt)



In [19]:
data_processed_for_training.to_csv('state_data_2015-2019_processed_training.csv', index = False)